In [5]:
using ProgressBars
using ITensors,ITensorMPS
using PastaQ
using RandomMeas

In [ ]:
#Perform RM in the first experiment
N = 6
ξ = siteinds("Qubit", N)

p,dp = 0.15,0.02
p_vec = [max(0, min(p + dp * randn(), 1)) for i in 1:N]
G = 1 .-p_vec/2

circuit = randomcircuit(N, depth=2)
states = ["Dn" for n in 1:N]
ψ0  = MPS(ComplexF64,ξ,states)

ρ0 = outer(ψ0',ψ0)
ρ0n = copy(ρ0)
Apply_depo_channel!(ρ0n,ξ,p_vec)

#
ρ = runcircuit(ρ0,circuit)
ρn = copy(ρ)
Apply_depo_channel!(ρn,ξ,p_vec)
#ρ = runcircuit(circuit;noise = noisemodel);
println("state prepared in the `experiment' 1 ")
#RM on a calibration state


nu=100 #Number of random unitaries
NM=10000 #Number of projective measurements per unitary

In [1]:
#Calibration experiment
Gs = zeros(Float64,N)
for r in ProgressBar(1:nu, printing_delay=2)
        u = get_rotations(ξ,1) #Haar rotations in A
        data = get_RandomMeas(ρ0n,u,NM)
        σu = apply(u,ρ0,apply_dag=true)
        #σu = rotate(ρ0,u)
        for i in 1:N
            P = get_Born(data[:,i:i],ξ[i:i])
            #P2 = get_Born_data_binary(datat2[:,i:i],ξ[i:i])
            #@show norm(P-P2)
            σui,ξi = reduce_dm(σu,collect(i:i))
            Pσ = get_Born(σui)
            Gs[i] += real((3*(P-Pσ)*Pσ)[]+1)/nu
            #Gs[i] += real((3*(P-Pσ)*Pσ)[]+1)/nu
        end
end
println(G)
println(Gs)

LoadError: UndefVarError: `N` not defined

In [2]:
u = Vector{Vector{ITensor}}()
data = zeros(Int8,nu,NM,N)
for r in ProgressBar(1:nu, printing_delay=2)
            push!(u,get_rotations(ξ,1)) #generate random unitary
            data[r,:,:] = get_RandomMeas(ρn,u[r],NM) #data acquisation in simulated quantum device
end

LoadError: UndefVarError: `ITensor` not defined

In [3]:
ITensors.disable_warn_order()
purity = zeros(Float64,N)
purity_nonmit = zeros(Float64,N)
purity_mit = zeros(Float64,N)

for NA in ProgressBar(1:N, printing_delay=2)
        purity[NA] = get_purity(ρ,collect(1:NA),ξ)
        uA = [ut[1:NA] for ut in u]
        purity_nonmit[NA] = get_purity_shadows(data[:,:,1:NA],uA,ξ[1:NA])
        purity_mit[NA] = get_purity_shadows(data[:,:,1:NA],uA,ξ[1:NA];G=Gs)

end

LoadError: UndefVarError: `ITensors` not defined

In [4]:
using Plots
plot(1:N,purity,label="exact")
plot!(1:N,purity_nonmit,label="shadows")
plot!(1:N,purity_mit,label="robust shadows")
xlabel!("partition")
ylabel!("purity")

LoadError: UndefVarError: `N` not defined